In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
# Activate mypy type checking and set exception reporting mode
%load_ext nb_mypy
%xmode Minimal

In [ ]:
import nbimporter # Activate import functionality for notebooks

# Implementierung eines Spieles
Dieses Notebook führt alle einzelnen Dateien und Klassen zusammen und implementiert den Spielablauf einer Partie. Die verwendeten AI-Versionen können dabei frei gewählt werden. Ein menschlicher Spieler ist ebenfalls möglich.

## Vorbereitung
Alle Notebooks wurden unter Python mit der Version `3.10.2` erstellt. Im Folgenden kann die Version überprüft werden


In [ ]:
from platform import python_version
print("Required Python version:", "3.10.2")
print("Your Python version:\t", python_version())

# Setup commands
# conda create -n chess python=3.10.2 -c conda-forge
# conda activate chess
# pip install -r src/requirements.txt
# conda install jupyter -c conda-forge
# jupyter notebook

# Import

Folgende Dateien werden in dieser Reihenfolge importiert:
- `AI-Base-Class.ipynb`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.
- `Exercise02AI`: Die zweite Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt.
- `Exercise03AI`: Die dritte Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `Simplified Evaluation Function` auswählt.
- `Exercise04AI`: Die vierte Version der AI, welche im Vergleich zur dritten Version mithilfe von Memoisierung beschleunigt wurde.
- `Exercise05AI`: HIER FEHLT EINE BESCHREIBUNG
- `Exercise06AI`: HIER FEHLT EINE BESCHREIBUNG
- `Exercise07AI`: HIER FEHLT EINE BESCHREIBUNG
- `HumanPlayer`: Ein Spieler welcher durch einen Menschen gesteuert werden kann.
- `StockfishPlayer`: Ein Spieler, welcher durch die Stockfish Engine gesteuert wird.

Anschließend wird das Spiel durchgeführt und auf der Festplatte als `.pgn`-Datei gespeichert.

In [ ]:
from AIBaseClass import ChessAI, State
from Exercise01AI import Exercise01AI
from Exercise02AI import Exercise02AI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI
from Exercise05AI import Exercise05AI
from Exercise06AI import Exercise06AI
from Exercise07AI import Exercise07AI
from Exercise07LiteAI import Exercise07LiteAI
from HumanPlayer import HumanPlayer
from StockfishPlayer import StockfishPlayer

## Speichern der Partie
Diese Funktion speichert eine gegebene Partie (repräsentiert durch ein gespieltes Board) auf der Festplatte ab. Hierbei werden sämtliche Züge, die Spielernamen und einige weitere Metriken gespeichert.

In [ ]:
def parse_player_name(player: ChessAI):
    """Parse the type of the current player and its name into a single string"""
    return f"{type(player).__name__} - {player.name}" 

In [ ]:
from datetime import datetime
import chess_custom as chess
import re
import os
from typing import Dict

def save_game(board: chess.Board, transitions: Dict[int, State], player1: ChessAI, player2: ChessAI, rep: str, duration: float, seed: int) -> None:
    """Saves last game to 'games/YYYY-mm-dd_HH-MM-SS.pgn' (based on current time)"""
    game = chess.pgn.Game.from_board(board)
    outcome = board.outcome(claim_draw=True)

    try:
        username = os.getlogin()
    except:
        username = "undefined"
    
    # set pgn headers
    game.headers["Event"] = "Chess-AI game"
    game.headers["Site"] = username
    game.headers["Date"] = datetime.now().strftime("%d.%m.%Y")
    game.headers["Round"] = rep
    game.headers["White"] = parse_player_name(player1)
    game.headers["Black"] = parse_player_name(player2)
    game.headers["Duration"] = str(duration)
    game.headers["Seed"] = str(seed)
    game.headers["Outcome"] = str(outcome.termination) if outcome else "Unknown"
    game.headers["CacheHits"] = str(getattr(player1, 'cache_hits', 0)) + "-" + str(getattr(player2, 'cache_hits', 0))
    game.headers["Depth"] = str(getattr(player1, 'DEPTH', "x")) + "-" + str(getattr(player2, 'DEPTH', "x"))
       
    # add transitions as comments to pgn file
    move_count = 0
    for node in game.mainline():
        move_count += 1
        state = transitions.get(move_count)
        if state:
            node.comment = str(state)
    
    # write game to pgn file
    filename = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    with open(f"../games/{filename}.pgn", 'w') as gamefile:
        # add a linebreak after each turn
        gamefile.write(re.sub(r" ([1-9][0-9]*)\.", r"\n\1.", str(game)))
        # add a newline to the end of the file
        gamefile.write("\n")

## Spielablauf (einzelnes Spiel)
Hier wird ein einzelnes neues Spiel initialisiert und anschließend gestartet. Die Funktion führt dabei nacheinander die Züge für die einzelnen Spieler aus und überwacht den aktuellen Spielzustand (State). Sobald ein Spieler den Endzustand erreicht hat wird das Spiel beendet. Zusätzlich werden aktuelle Statistiken ausgegeben.


In [ ]:
from IPython.display import clear_output, display
import time
from typing import Tuple

def run_single_game(player1: ChessAI, player2: ChessAI, start_time: float) -> Tuple[chess.Board, Dict[int, State]]:
    """Start a new chess game"""
    is_human_run = not bool(os.environ.get("CI", False))
    # Create a new board
    board = chess.Board()
    players = [player1, player2]

    # All players start with opening game
    previous_state = {player.name: State.OPENING_GAME for player in players}
    transitions = { 1: State.OPENING_GAME, 2: State.OPENING_GAME }
    move_count = 0

    # Show board before first move (for human player)
    if is_human_run:
        display(board)
    
    # Statistics variable
    statistics = ""

    # Play game
    while True:
        for player in players:
            # Make next move
            player.make_turn(board)
            move_count += 1
            
            # Print board
            if is_human_run:
                clear_output(wait=True)
                display(board)
                print(f'Current state ({player.name}): {player.state}\n')
                print(statistics, end='')
            else:
                print(f"{type(player).__name__} ({player.state}) ==> '{board.fen()}'")

            # Watch game progress slowly
            # time.sleep(0.1)

            # Log state transitions
            if previous_state[player.name] != player.state:
                # Update state
                previous_state[player.name] = player.state
                
                # Pause at transition
                # input("Press enter to continue")

                # Exit if game has finished
                if player.state == State.FINISHED:
                    outcome = board.outcome(claim_draw=True)
                    assert outcome, f'Outcome is None in State.FINISHED and FEN {board.fen()}!'
                    if outcome.winner == chess.WHITE:
                        print(f"\nWhite won the match due to {str(outcome.termination)} with result {outcome.result()}!")
                    elif outcome.winner == chess.BLACK:
                        print(f"\nBlack won the match due to {str(outcome.termination)} with result {outcome.result()}!")
                    else:
                        print(f"\nThe game is a remis due to {str(outcome.termination)} with result {outcome.result()}!")
                    # State.FINISHED is only detected AFTER the finishing move
                    transitions[move_count - 1] = player.state
                    return board, transitions
                else:
                    transitions[move_count] = player.state

## Spielstart (mehrere Spiele)
Hier wird initial der Seed gesetzt und anschließend die gewünschte Anzahl an Spielen ausgeführt. Pro Spiel wird die Zeit gemessen. Alle Spiele werden anschließend abgespeichert.

In [ ]:
def run_games(player1: ChessAI, player2: ChessAI, repetitions: int, seed: int):
    seed = int(os.environ.get("CHESS_AI_SEED", seed))
    ChessAI.random.seed(seed)
    repetitions = int(os.environ.get("CHESS_AI_REPETITIONS", repetitions))

    for rep in range(repetitions):
        print(f"Running game {rep+1} of {repetitions} with seed {seed}")
        print(f"Player {type(player1).__name__} (white) vs {type(player2).__name__} (black)")
        t_start = time.time()
        board, transitions = run_single_game(player1, player2, t_start)
        t_end = time.time()
        rep_str = f"{rep + 1}/{repetitions}"
        save_game(board, transitions, player1, player2, rep_str, t_end - t_start, seed)
        player1.reset()
        player2.reset()

    print(f"Used seed {seed} with {repetitions} repetitions")

## Spielstart
Hier werden die Spieler definiert und die gewünschte Anzahl an Spielen gestartet.

In [ ]:
%%time
# Create Players
player1 = Exercise07LiteAI(player_name="White")
player2 = Exercise07AI(player_name="Black")

run_games(player1, player2, repetitions=1, seed=13)